In [1]:
# Imports required libaries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import pickle
import re

import spacy
from nltk.corpus import stopwords
import nltk
from collections import Counter
from sklearn.feature_extraction._stop_words import ENGLISH_STOP_WORDS
# nltk.download('names')
# nltk.download('stopwords')

In [2]:
# Load spacy 
nlp = spacy.load("en_core_web_sm")

In [3]:
# Load the data frame
sc_india = pd.read_csv('Supreme_Court_cases_India_all.csv')
sc_india.drop(sc_india.filter(regex="Unname"),axis=1, inplace=True)
sc_india.head()

,Case Title,Year,Author,Judges,Document Id
0,Hardeep Singh vs State Of Punjab & Ors on 10 J...,1947,. B Chauhan,"P Sathasivam, B.S. Chauhan, Ranjana Prakash D...",52754564
1,Sanjay Kumar vs State Of Bihar & Anr on 28 Jan...,1947,…………......................J.,"B.S. Chauhan, J. Chelameswar, M.Y. Eqbal",130498285
2,State Of T.Nadu Tr.Insp.Of Police vs N Suresh ...,1947,C K Prasad,"Chandramauli Kr. Prasad, M.Y. Eqbal",23015624
3,Union Of India & Ors vs Vasavi Co-Op. Housing ...,1947,…..………………………J.,"K.S. Radhakrishnan, A.K. Sikri",175295508
4,"Syed Sadiq Etc vs Divisional Manager,United In...",1947,………………………………………………………………………J.,"Sudhansu Jyoti Mukhopadhaya, V. Gopala Gowda",140442067


In [4]:
sc_india.shape

(53423, 5)

In [6]:
# Logic to fetch the paragraph from the site
website = 'https://indiankanoon.org/doc/130498285/'
response = requests.get(website)
soup = BeautifulSoup(response.content, 'html.parser')
# print(soup)
for i in range(1, 1000):
        try:
            para = soup.find('p',  {'id' : 'p_'+str(i)}).get_text().strip()
            doc_para_list.append(para)
            # print(para)
        except:
            break

In [7]:
# doc_list contains a strings, where each string belongs to a particular document
doc_list = []
for i in range(20):           # Only 20 Documents are fetched for testing purpose
    website = 'https://indiankanoon.org/doc/'+str(sc_india['Document Id'][i])+'/'
    response = requests.get(website)
    soup = BeautifulSoup(response.content, 'html.parser')
    doc_m = ""
    for i in range(1, 1000):
        try:
            para = soup.find('p',  {'id' : 'p_'+str(i)}).get_text().strip()
            doc_m = doc_m + para
        except:
            break
    doc_list.append(doc_m.casefold())

In [9]:
# doc_list[0]

In [22]:
# Get all the possible stop words using available libraries
# Default stop word by spacy

# Remark - All stopwords are converted to lower case

spacy_stop = nlp.Defaults.stop_words

# Names by nltk
names = nltk.corpus.names
male_names = names.words('male.txt')
male_names = [name.casefold() for name in male_names]
female_names = names.words('female.txt')
female_names = [name.casefold() for name in female_names]

# Months are also stop words in our case
months = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august' 'september', 'october', 'november',
          'december']
# Other most common less meaningful words
more_words = ['court', 'courts', 'justice', 'appeals','appeal', 'argued', 'argue', 'decide', 'rptr', 'nervine', 'pp','fd', 'states',
              'supreme', 'opinion' ,'cheif', 'el', 'op', 'quotation', "n't" , 'a', 'aan', 'aba', 'aand', 'of', 'm', 'u',
              'f', 'j', 'juan', 'ca', 'aa', 'aaa','aab', 'aabd', 'aac',  'aag', 'aai', 'aaii', 'aaiii', 'aalthough', 'judge', 
              'author', 'state', 'country', 'india']

evenmore_words = ['join', 'seek','hummel', 'note', 'curiam', 'mosk', 'pd', 'rhino', 'misc', 'assistant', 'whereon', 'dismiss',
                  'sod', 'vote', 'present', 'entire', 'frankfurter', 'leave', 'concur', 'entire', 'mootness', 'track',
                  'constitution', 'jj', 'stat', 'messes', 'like', 'rev', 'trans', 'bra', 'teller', 'lead', 'cf', 'cca', 
                  'stucky', 'aver',"united", "dissent", "footnote","brief", "decision", "member", "curiam", "dismiss", 
                  "note", "affirm", "question", "usc", "file", "district", "circuit", "mr", "law", "quoting", "omit", 'period',
                  "amendment","internal", "slip", 'omitted', 'suit' ,'lawsuit', 'marks', "jr", "findlaw ", "href", "defendant",
                  "judge", "rule", "claim", "comussupremecourt", "petitioner", "act", "federal", "statute", "government", "right",
                  'bench', 'crpc', 'section']


In [23]:
# State names as well as its capitals

website = 'https://en.wikipedia.org/wiki/List_of_state_and_union_territory_capitals_in_India'
response = requests.get(website)
soup = BeautifulSoup(response.content, 'html.parser')

states_table = soup.find_all('table', {'class' : 'wikitable'})[0]
states_table = states_table.find_all('tr')

states = []
executive_capital = []
legislative_capital = []
judicial_capital = []

for i in range(1, len(states_table)):
    state = states_table[i].find('th', {'scope':'row'}).get_text().strip()
    states.append(state.casefold())
    
    e_capital = re.sub(r'\[\w\]|(\(Summer\))|(\(Winter\))', '', states_table[i].find_all('td')[1].get_text()).strip()
    executive_capital.append(e_capital.casefold())
    
    l_capital = re.split(r'\s+',(re.sub(r'(\[\w\])|(\(Summer\))|(\(Winter\))', '', states_table[i].find_all('td')[2].get_text())).strip())
    for l in l_capital : legislative_capital.append(l.casefold()) 
        
    j_capital = re.sub(r'\[\w\]', '', states_table[i].find_all('td')[3].get_text()).strip()
    judicial_capital.append(j_capital.casefold())
    
union_territories_table = soup.find_all('table', {'class' : 'wikitable'})[1]
union_territories_table = union_territories_table.find_all('tr')

for i in range(1, len(union_territories_table)):
    ut = union_territories_table[i].find_all('td')[1].get_text().strip()
    states.append(ut.casefold())
    
    e_capital = re.split(r'\s+',(re.sub(r'(\[\w\])|(\(Summer\))|(\(Winter\))', '', union_territories_table[i].find_all('td')[2].get_text())).strip())
    for e in e_capital : 
        if e not in executive_capital:
            executive_capital.append(e.casefold())
            
    l_capital = re.split(r'\s+',(re.sub(r'(\[\w\])|(\(Summer\))|(\(Winter\))', '', union_territories_table[i].find_all('td')[3].get_text()).strip()))
    for l in l_capital : 
        if l != '-' and l not in legislative_capital : 
            legislative_capital.append(l.casefold()) 
            
    j_capital = re.split(r'\s+',(re.sub(r'(\[\w\])|(\(Summer\))|(\(Winter\))', '', union_territories_table[i].find_all('td')[4].get_text())).strip())
    for j in j_capital : 
        if j not in judicial_capital:
            judicial_capital.append(j.casefold())


In [24]:
print(executive_capital)

['visakhapatnam', 'itanagar', 'dispur', 'patna', 'naya raipur', 'panaji', 'gandhinagar', 'chandigarh', 'shimla', 'ranchi', 'bengaluru', 'thiruvananthapuram', 'bhopal', 'mumbai', 'imphal', 'shillong', 'aizawl', 'kohima', 'bhubaneswar', 'chandigarh', 'jaipur', 'gangtok', 'chennai', 'hyderabad', 'agartala', 'lucknow', 'gairsain   (summer)\ndehradun (winter)', 'kolkata', 'port', 'blair', 'chandigarh', 'daman', 'new', 'delhi', 'srinagar', '(summer)', 'jammu', '(winter)', 'leh', 'kargil', 'kavaratti', 'pondicherry']


In [25]:
print(legislative_capital)

['amaravati', 'itanagar', 'dispur', 'patna', 'raipur', 'porvorim', 'gandhinagar', 'chandigarh', 'shimla', 'dharamsala', 'ranchi', 'bengaluru', 'thiruvananthapuram', 'bhopal', 'mumbai', 'nagpur', 'imphal', 'shillong', 'aizawl', 'kohima', 'bhubaneswar', 'chandigarh', 'jaipur', 'gangtok', 'chennai', 'hyderabad', 'agartala', 'lucknow', 'gairsain', '(summer)dehradun', '(winter)', 'kolkata', 'new', 'delhi', 'srinagar', '(summer)', 'jammu', 'pondicherry']


In [26]:
print(judicial_capital)

['kurnool', 'guwahati', 'guwahati', 'patna', 'bilaspur', 'mumbai', 'ahmedabad', 'chandigarh', 'shimla', 'ranchi', 'bengaluru', 'kochi', 'jabalpur', 'mumbai', 'imphal', 'shillong', 'guwahati', 'guwahati', 'cuttack', 'chandigarh', 'jodhpur', 'gangtok', 'chennai', 'hyderabad', 'agartala', 'prayagraj', 'nainital', 'kolkata', 'kolkata', 'chandigarh', 'mumbai', 'new', 'delhi', 'srinagar', '(summer)', 'jammu', '(winter)', 'srinagar', 'jammu', 'kochi', 'chennai']


In [36]:
# sc_india['Judges'].isnull().sum()
# sc_india['Author'].isnull().sum()
# sc_india['Judges'].nunique()
# sc_india['Author'].nunique()

In [10]:
# Get the justice as well as author names
judges_doc = list(sc_india['Judges'].unique())
authors_doc = list(sc_india['Author'].unique())
judges = []
authors = []
for judges_all in judges_doc:
    try:
        judges_split = judges_all.split(',')
        for judge in judges_split:
            if judge not in judges :
                judges.append(judge.casefold())
    except:
        continue
        

for authors_all in authors_doc:
    try:
        authors_split = authors_all.split(',')
        for author in authors_split:
            if author not in authors :
                authors.append(author.casefold())
    except:
        continue

In [13]:
# print(judges)
# print(authors)

In [20]:
# Get Indian names as well
indian_male = pd.read_csv('Indian-Male-Names.csv')
names = list(indian_male.name)
indian_male_name = []
for name in names:
    try:
        for nm in name.split(' '):
            indian_male_name.append(nm.replace('@', ''))
    except:
        continue
        
indian_male_name = set(indian_male_name)
# print(indian_male_name)

indian_female = pd.read_csv('Indian-Female-Names.csv')
names = list(indian_female.name)
indian_female_name = []
for name in names:
    try:
        for nm in name.split(' '):
            indian_female_name.append(nm.replace('@', ''))
    except:
        continue
        
indian_female_name = set(indian_female_name)
# print(indian_female_name)

In [27]:
STOPLIST = set( stopwords.words('english')  + list(ENGLISH_STOP_WORDS) + judges + authors + list(female_names) + list(male_names) 
                + list(spacy_stop) + months + more_words + evenmore_words + states + executive_capital + list(indian_male_name) + 
                list(indian_female_name) + legislative_capital + judicial_capital)
STOPLIST = set(stopword.lower() for stopword in STOPLIST)

In [10]:
len(STOPLIST)

20762

In [14]:
doc_list[0]

'dr. b.s. chauhan, j.1.   this reference before us arises out of a variety of views  having\n      been expressed by this court and several high courts of the country on\n      the scope and extent of the powers of the courts  under  the  criminal\n      justice system to arraign any person as an accused during  the  course\n      of inquiry or trial as contemplated under section 319 of the  code  of\n      criminal procedure, 1973 (hereinafter referred to as the `cr.p.c.’).2.    the initial reference was made by a two-judge bench  vide  order\n      dated 7.11.2008 in the leading case of hardeep singh (crl. appeal  no.\n      1750 of 2008)  where noticing the conflict between  the  judgments  in\n      the case of rakesh v. state of haryana, air 2001 sc 2521; and  a  two-\n      judge bench decision in the case of mohd. shafi v. mohd. rafiq & anr.,\n      air 2007 sc 1899, a doubt was expressed about the correctness  of  the\n      view in the case of mohd. shafi (supra). the doubts a

In [15]:
# clean the text with some overused words
def clean(text):
#   print(type(text))
    text = text.replace("argued:", " ")
    text = text.replace("supreme court of india", " ")
    text = text.replace("constitution of india", " ")
    text = text.replace("\\n", " ")       
    text = re.sub(r'\(\w+\)', '', text)  # remove all present the content in the brackets
    text = re.sub(r'[^a-z^ ]', '', text)  # remove all chars other than a-z and space
    return text

In [16]:
clean_doc = clean(doc_list[0])

In [17]:
clean_doc

'dr bs chauhan j   this reference before us arises out of a variety of views  having      been expressed by this court and several high courts of the country on      the scope and extent of the powers of the courts  under  the  criminal      justice system to arraign any person as an accused during  the  course      of inquiry or trial as contemplated under section  of the  code  of      criminal procedure  hereinafter referred to as the crpc    the initial reference was made by a twojudge bench  vide  order      dated  in the leading case of hardeep singh crl appeal  no       of   where noticing the conflict between  the  judgments  in      the case of rakesh v state of haryana air  sc  and  a  two      judge bench decision in the case of mohd shafi v mohd rafiq  anr      air  sc  a doubt was expressed about the correctness  of  the      view in the case of mohd shafi  the doubts as categorised  in      paragraphs  and  of the reference order led to the framing of  two      questions 

In [31]:
# Tokenization of the doc
def spacy_tokenizer(sentence):
    tokens = nlp(sentence)
    tokens = [tok.lemma_.lower().strip() if tok.lemma_ != "-PRON-" else tok.lower_ for tok in tokens]
    tokens = [tok for tok in tokens if (len(tok)>1)]
    tokens = [tok for tok in tokens if tok not in STOPLIST] 
    return tokens


In [32]:
token_doc = spacy_tokenizer(clean_doc)

In [34]:
# token_doc

In [40]:
# if 'chauhan' in token_doc:
#     print('yes')
# else:
#     print('no')

In [36]:
word_count = Counter(token_doc)

In [37]:
# Create a dic of the word_count for each doc
def word_count_dic(doc_id, token_doc):
    word_count = Counter(token_doc)
    sort_word_count = sorted(word_count.items(), key= lambda x:x[1], reverse=True)
    return {doc_id : sort_word_count}

In [38]:
word_count_dic(sc_india['Document Id'][i], token_doc)

{63441281: [('evidence', 133),
  ('trial', 102),
  ('accuse', 90),
  ('power', 86),
  ('stage', 81),
  ('inquiry', 67),
  ('exercise', 61),
  ('material', 58),
  ('sc', 55),
  ('air', 54),
  ('hold', 53),
  ('offence', 52),
  ('word', 52),
  ('provision', 43),
  ('magistrate', 37),
  ('session', 35),
  ('summon', 34),
  ('proceed', 32),
  ('anr', 31),
  ('use', 31),
  ('record', 31),
  ('appear', 26),
  ('cognizance', 25),
  ('try', 24),
  ('view', 23),
  ('charge', 23),
  ('invoke', 22),
  ('basis', 22),
  ('purpose', 22),
  ('mean', 22),
  ('discharge', 20),
  ('fact', 20),
  ('commit', 19),
  ('commence', 19),
  ('require', 19),
  ('come', 18),
  ('chargesheet', 18),
  ('witness', 18),
  ('refer', 17),
  ('investigation', 17),
  ('order', 16),
  ('add', 16),
  ('legislature', 16),
  ('face', 16),
  ('issue', 15),
  ('committal', 15),
  ('time', 15),
  ('satisfaction', 15),
  ('collect', 15),
  ('prima', 15),
  ('facie', 15),
  ('till', 14),
  ('prosecution', 14),
  ('reference', 13)

In [65]:
# Create a DataFrame to store the token along with it's id
token_sc_india = pd.DataFrame(columns=['Document Id', 'Tokens'])
word_count_list = []

for x, y in enumerate(doc_list):
    doc_id = sc_india['Document Id'][x]
    clean_doc = clean(y)    
    token_doc = spacy_tokenizer(clean_doc)
    token_sc_india = token_sc_india.append({ 'Document Id' : doc_id, 'Tokens' : token_doc}, ignore_index = True)
    word_count_list.append(word_count_dic(doc_id, token_doc))

In [39]:
# word_count_list

In [66]:
token_sc_india.head()

,Document Id,Tokens
0,52754564,"[bs, reference, arise, variety, view, express,..."
1,130498285,"[special, petition, impugn, judgment, order, d..."
2,23015624,"[chandramauli, jcriminal, special, petitioncrl..."
3,175295508,"[ks, radhakrishnan, vasavi, coop, housing, soc..."
4,140442067,"[gopala, gowda, jleave, appellant, correctness..."


In [78]:
sc_india.head()

,Case Title,Year,Author,Judges,Document Id
0,Hardeep Singh vs State Of Punjab & Ors on 10 J...,1947,. B Chauhan,"P Sathasivam, B.S. Chauhan, Ranjana Prakash D...",52754564
1,Sanjay Kumar vs State Of Bihar & Anr on 28 Jan...,1947,…………......................J.,"B.S. Chauhan, J. Chelameswar, M.Y. Eqbal",130498285
2,State Of T.Nadu Tr.Insp.Of Police vs N Suresh ...,1947,C K Prasad,"Chandramauli Kr. Prasad, M.Y. Eqbal",23015624
3,Union Of India & Ors vs Vasavi Co-Op. Housing ...,1947,…..………………………J.,"K.S. Radhakrishnan, A.K. Sikri",175295508
4,"Syed Sadiq Etc vs Divisional Manager,United In...",1947,………………………………………………………………………J.,"Sudhansu Jyoti Mukhopadhaya, V. Gopala Gowda",140442067


In [79]:
sc_india.Year.unique()

array([1947, 1948, 1949, 1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957,
       1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968,
       1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979,
       1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990,
       1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001,
       2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,
       2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021], dtype=int64)

In [ ]:
# Finally apply all the functions and fetch a sorted data

In [41]:
sc_india[(sc_india.Year >= 2001) ].shape

(25007, 5)

In [42]:
sc_india_recent = sc_india[sc_india.Year >= 2001].reset_index()   
sc_india_recent.drop(['index'], axis = 1, inplace = True) 

In [43]:
sc_india_recent.shape

(25007, 5)

In [ ]:
sc_india_recent.to_csv('Supreme_Court_cases_India_above_1999.csv')

In [ ]:
# sc_india_recent = pd.read_csv('Supreme_Court_cases_India_above_1999.csv')
# sc_india_recent.drop(sc_india_recent.filter(regex="Unname"),axis=1, inplace=True)

In [45]:
# The dataset is divides into two parts 1 : (2001 - 2005), (2006 - 2021)
sc_india_recent[(sc_india_recent.Year >= 2001) & (sc_india_recent.Year <= 2005)].shape

(6030, 5)

In [46]:
# Consider sc_india_recent dataframe only from year 2001 to 2006 
sc_india_recent = sc_india[(sc_india.Year >= 2001) & (sc_india.Year <= 2005) ].reset_index()   
sc_india_recent.drop(['index'], axis = 1, inplace = True) 

In [47]:
sc_india_recent.head()

,Case Title,Year,Author,Judges,Document Id
0,Smt. Kaushnuma Begum And Ors vs The New India ...,2001,NaN,"K.T.Thomas, R.P.Sethi",484625
1,Dalmia Cement (Bharat) Ltd vs M/S.Galaxy Trade...,2001,Sethi,"R.P.Sethi, K.T.Thomas",60864
2,Suresh Chand Jain vs State Of Madhya Pradesh &...,2001,NaN,"K.T.Thomas, R.P.Sethi",1373794
3,West Bengal State Electricity ... vs Patel Eng...,2001,S S Quadri,"S.S.M.Quadri, S.N.Phukan",55804
4,Ragu Thilak D. John vs S.Rayappan & Others on ...,2001,Sethi,"K.T. Thomas, R.P. Sethi.",301256


In [48]:
sc_india_recent.tail()

,Case Title,Year,Author,Judges,Document Id
6025,State Of M.P. And Ors vs Onkar Prasad Patel on...,2005,A Pasayat,"Arijit Pasayat, Tarun Chatterjee",1735977
6026,"Harjit Singh vs State Of Punjab on 8 December,...",2005,NaN,"S.B. Sinha, P.P. Naolekar",366871
6027,Siddartha Tubes Ltd vs Commissioner Of Central...,2005,Kapadia,"Ashok Bhan, S.H. Kapadia",1684677
6028,Madan Lal And Anr. vs Bal Krishan And Ors. on ...,2005,A Pasayat,"A Pasayat, T Chatterjee",467623
6029,A. Maimoona vs State Of Tamil Nadu & Ors on 16...,2005,P Balasubramanyan,"S.B. Sinha, P.K. Balasubramanyan",1223961


In [145]:
# Create a doc_list that contains a corpus of document
doc_list = []
for i in range(sc_india_recent.shape[0]):  
    website = 'https://indiankanoon.org/doc/'+str(sc_india_recent['Document Id'][i])+'/'
    response = requests.get(website)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    doc_m = ""
    print(f"{i} -> {sc_india_recent['Document Id'][i]}")
    
    for i in range(1, 1000):
        try:
            para = soup.find('p',  {'id' : 'p_'+str(i)}).get_text().strip()
            doc_m = doc_m + para
        except:
            break
            
    doc_list.append(doc_m.casefold())

0 -> 484625
1 -> 60864
2 -> 1373794
3 -> 55804
4 -> 301256
5 -> 342526
6 -> 315034
7 -> 952963
8 -> 1919674
9 -> 339039
10 -> 828091
11 -> 30721
12 -> 330864
13 -> 1871672
14 -> 1444315
15 -> 1070947
16 -> 171884
17 -> 1703868
18 -> 864906
19 -> 1087177
20 -> 88397
21 -> 577272
22 -> 952306
23 -> 1392246
24 -> 302447
25 -> 723319
26 -> 316662
27 -> 869424
28 -> 161755
29 -> 32069
30 -> 1072924
31 -> 581738
32 -> 492319
33 -> 1404079
34 -> 628924
35 -> 29716
36 -> 1796201
37 -> 526619
38 -> 1787248
39 -> 165700
40 -> 511928
41 -> 1573419
42 -> 1825956
43 -> 1058158
44 -> 754035
45 -> 1975354
46 -> 672732
47 -> 1981933
48 -> 108824706
49 -> 104106942
50 -> 1483568
51 -> 619652
52 -> 849832
53 -> 559864
54 -> 492051
55 -> 1466071
56 -> 1581335
57 -> 1622700
58 -> 124441393
59 -> 1815829
60 -> 1799063
61 -> 1974403
62 -> 404475
63 -> 96514934
64 -> 1269481
65 -> 465907
66 -> 1065048
67 -> 805009
68 -> 1178541
69 -> 48569
70 -> 696401
71 -> 849501
72 -> 1242489
73 -> 643598
74 -> 4960191
75

569 -> 441929
570 -> 1974324
571 -> 1206100
572 -> 1701224
573 -> 1917283
574 -> 1389589
575 -> 1791620
576 -> 1600445
577 -> 1766988
578 -> 605834
579 -> 1323993
580 -> 1483262
581 -> 1815324
582 -> 1706713
583 -> 641383
584 -> 870673
585 -> 989798
586 -> 592428
587 -> 454130
588 -> 1973429
589 -> 1130828
590 -> 53461189
591 -> 1030147
592 -> 406616
593 -> 840688
594 -> 1248667
595 -> 1691248
596 -> 455820
597 -> 1865386
598 -> 893566
599 -> 839959
600 -> 902306
601 -> 1442809
602 -> 1242357
603 -> 147864248
604 -> 253640
605 -> 1138910
606 -> 1945324
607 -> 1852811
608 -> 791266
609 -> 735843
610 -> 199374
611 -> 1752197
612 -> 739978
613 -> 162668
614 -> 250025
615 -> 97268
616 -> 1132839
617 -> 1679705
618 -> 1117516
619 -> 596603
620 -> 812915
621 -> 865927
622 -> 874542
623 -> 1439141
624 -> 1670628
625 -> 1210590
626 -> 135784847
627 -> 1725787
628 -> 656475
629 -> 143274
630 -> 1352674
631 -> 1665059
632 -> 795107
633 -> 515955
634 -> 1928163
635 -> 678728
636 -> 139810459
637 

1122 -> 147831146
1123 -> 138050962
1124 -> 1027994
1125 -> 586957
1126 -> 1737583
1127 -> 1479522
1128 -> 1299452
1129 -> 673494
1130 -> 139299795
1131 -> 137379
1132 -> 628651
1133 -> 841113
1134 -> 1928188
1135 -> 1918955
1136 -> 1085133
1137 -> 261589
1138 -> 1915429
1139 -> 176692787
1140 -> 276646
1141 -> 853433
1142 -> 1669097
1143 -> 1654170
1144 -> 1736083
1145 -> 1248305
1146 -> 1800100
1147 -> 892046
1148 -> 1890435
1149 -> 1949357
1150 -> 1919444
1151 -> 74318473
1152 -> 96789
1153 -> 433507
1154 -> 1277242
1155 -> 1013105
1156 -> 1241417
1157 -> 761005
1158 -> 149756
1159 -> 1808484
1160 -> 765135
1161 -> 40737166
1162 -> 104928945
1163 -> 196100928
1164 -> 783492
1165 -> 712499
1166 -> 747596
1167 -> 34445
1168 -> 1829378
1169 -> 428148
1170 -> 309619
1171 -> 202973
1172 -> 45961
1173 -> 325522
1174 -> 1904257
1175 -> 1386587
1176 -> 1014506
1177 -> 315007
1178 -> 1868789
1179 -> 1651003
1180 -> 1957714
1181 -> 18551101
1182 -> 50693703
1183 -> 1768336
1184 -> 255258
1185

1648 -> 111092
1649 -> 1933466
1650 -> 27901
1651 -> 409545
1652 -> 76327630
1653 -> 1412571
1654 -> 144524555
1655 -> 1798667
1656 -> 1349336
1657 -> 1992311
1658 -> 1189438
1659 -> 208104
1660 -> 1234243
1661 -> 373674
1662 -> 1541035
1663 -> 1842561
1664 -> 637121
1665 -> 1765136
1666 -> 1257127
1667 -> 1386978
1668 -> 1670737
1669 -> 1444013
1670 -> 1020761
1671 -> 212701
1672 -> 1132466
1673 -> 103664
1674 -> 240136
1675 -> 1310828
1676 -> 1382201
1677 -> 181338271
1678 -> 317443
1679 -> 818332
1680 -> 1861993
1681 -> 1246989
1682 -> 296919
1683 -> 1622732
1684 -> 1493294
1685 -> 849428
1686 -> 200529
1687 -> 288213
1688 -> 816027
1689 -> 1877867
1690 -> 1318795
1691 -> 160887
1692 -> 48351
1693 -> 339045
1694 -> 100834628
1695 -> 1573754
1696 -> 1529194
1697 -> 1224351
1698 -> 1584515
1699 -> 1180876
1700 -> 1212086
1701 -> 554694
1702 -> 516669
1703 -> 1201968
1704 -> 854624
1705 -> 1724440
1706 -> 924107
1707 -> 711454
1708 -> 424832
1709 -> 1389120
1710 -> 1128474
1711 -> 1131

2174 -> 1911675
2175 -> 344330
2176 -> 1063572
2177 -> 1020649
2178 -> 570393
2179 -> 976161
2180 -> 408620
2181 -> 344937
2182 -> 462447
2183 -> 1854196
2184 -> 215097
2185 -> 921223
2186 -> 226350
2187 -> 342770
2188 -> 1539576
2189 -> 829043
2190 -> 1180569
2191 -> 609016
2192 -> 30094296
2193 -> 1007126
2194 -> 1847321
2195 -> 1287030
2196 -> 1458298
2197 -> 1868108
2198 -> 186697
2199 -> 806787
2200 -> 1926604
2201 -> 852956
2202 -> 1801195
2203 -> 1200609
2204 -> 1295690
2205 -> 586656
2206 -> 25199
2207 -> 1754001
2208 -> 14169
2209 -> 839516
2210 -> 427559
2211 -> 137487
2212 -> 1132527
2213 -> 1090819
2214 -> 1978077
2215 -> 1806497
2216 -> 1292274
2217 -> 634300
2218 -> 1002182
2219 -> 248422
2220 -> 40205
2221 -> 234299
2222 -> 20185201
2223 -> 518973
2224 -> 1746530
2225 -> 127004
2226 -> 95490
2227 -> 1440508
2228 -> 770879
2229 -> 1209191
2230 -> 1200520
2231 -> 393527
2232 -> 1596666
2233 -> 308516
2234 -> 1232029
2235 -> 665015
2236 -> 1934517
2237 -> 1727179
2238 -> 69

...

3731 -> 133992
3732 -> 11166039
3733 -> 138799331
3734 -> 895738
3735 -> 1546846
3736 -> 1728059
3737 -> 1676807
3738 -> 99628
3739 -> 905647
3740 -> 83284911
3741 -> 475850
3742 -> 1283428
3743 -> 300166
3744 -> 1724715
3745 -> 143699
3746 -> 1181002
3747 -> 146444
3748 -> 14299467
3749 -> 62010481
3750 -> 1295003
3751 -> 1629851
3752 -> 619838
3753 -> 1167322
3754 -> 65801028
3755 -> 976548
3756 -> 146455
3757 -> 1550126
3758 -> 301898
3759 -> 1938735
3760 -> 1583561
3761 -> 1943124
3762 -> 789051
3763 -> 1740690
3764 -> 110957682
3765 -> 761884
3766 -> 955330
3767 -> 1598563
3768 -> 38394
3769 -> 1870380
3770 -> 1549187
3771 -> 449895
3772 -> 520092
3773 -> 1640713
3774 -> 1097981
3775 -> 434621
3776 -> 1812977
3777 -> 622430
3778 -> 255210
3779 -> 1419648
3780 -> 1660548
3781 -> 350207
3782 -> 1498806
3783 -> 1976638
3784 -> 354829
3785 -> 686974
3786 -> 752130
3787 -> 851066
3788 -> 548162
3789 -> 943151
3790 -> 1332816
3791 -> 512571
3792 -> 870905
3793 -> 1582820
3794 -> 1957755

4253 -> 917436
4254 -> 34492610
4255 -> 151624502
4256 -> 477919
4257 -> 186696191
4258 -> 33643539
4259 -> 421171
4260 -> 74242914
4261 -> 44195342
4262 -> 1111437
4263 -> 185121721
4264 -> 139424125
4265 -> 129292798
4266 -> 105430
4267 -> 1059476
4268 -> 1876565
4269 -> 321019
4270 -> 1418721
4271 -> 1704181
4272 -> 1445805
4273 -> 231341
4274 -> 1290534
4275 -> 19951
4276 -> 122509
4277 -> 1293440
4278 -> 573333
4279 -> 1570352
4280 -> 1916813
4281 -> 1290723
4282 -> 1923755
4283 -> 1069885
4284 -> 511501
4285 -> 1216036
4286 -> 1951229
4287 -> 1084188
4288 -> 1497720
4289 -> 627760
4290 -> 1684919
4291 -> 225599
4292 -> 1783984
4293 -> 160789
4294 -> 1612780
4295 -> 1926027
4296 -> 1867627
4297 -> 496676
4298 -> 1618302
4299 -> 853716
4300 -> 1515604
4301 -> 492377
4302 -> 196302305
4303 -> 176647548
4304 -> 1072861
4305 -> 985284
4306 -> 766886
4307 -> 1615098
4308 -> 124279
4309 -> 853784
4310 -> 1721619
4311 -> 80804
4312 -> 240789
4313 -> 634266
4314 -> 441874
4315 -> 1864006


4778 -> 524281
4779 -> 220094
4780 -> 325096
4781 -> 783058
4782 -> 1948292
4783 -> 229208
4784 -> 803726
4785 -> 298579
4786 -> 765792
4787 -> 770272
4788 -> 648736
4789 -> 841129
4790 -> 1778819
4791 -> 1068747
4792 -> 161861178
4793 -> 947
4794 -> 712234
4795 -> 1436672
4796 -> 1332409
4797 -> 748814
4798 -> 1628531
4799 -> 982131
4800 -> 1583669
4801 -> 1203458
4802 -> 1423669
4803 -> 1732268
4804 -> 518831
4805 -> 1924157
4806 -> 607625
4807 -> 1147175
4808 -> 1994838
4809 -> 855361
4810 -> 970620
4811 -> 1729858
4812 -> 1177323
4813 -> 168545311
4814 -> 210097
4815 -> 49133735
4816 -> 1756632
4817 -> 1265820
4818 -> 348899
4819 -> 196226299
4820 -> 1501179
4821 -> 558373
4822 -> 149012918
4823 -> 1604025
4824 -> 818083
4825 -> 119042
4826 -> 198772887
4827 -> 64772
4828 -> 264557
4829 -> 150881
4830 -> 617135
4831 -> 1198044
4832 -> 1787277
4833 -> 1310174
4834 -> 1838507
4835 -> 12214027
4836 -> 342073
4837 -> 1409685
4838 -> 1300862
4839 -> 1463985
4840 -> 580863
4841 -> 166174

5300 -> 567128
5301 -> 466093
5302 -> 32662
5303 -> 1084013
5304 -> 1246724
5305 -> 527344
5306 -> 921001
5307 -> 790630
5308 -> 866941
5309 -> 1074322
5310 -> 910940
5311 -> 1357418
5312 -> 1534452
5313 -> 1589069
5314 -> 73715021
5315 -> 182991489
5316 -> 748067
5317 -> 1594596
5318 -> 1189023
5319 -> 833158
5320 -> 1561222
5321 -> 942439
5322 -> 1972239
5323 -> 899528
5324 -> 1915099
5325 -> 301445
5326 -> 527656
5327 -> 1430296
5328 -> 852405
5329 -> 1221178
5330 -> 115310075
5331 -> 1617880
5332 -> 391625
5333 -> 1338744
5334 -> 1758778
5335 -> 649814
5336 -> 948361
5337 -> 1897303
5338 -> 1497806
5339 -> 1951051
5340 -> 1078624
5341 -> 1527139
5342 -> 1184356
5343 -> 1393826
5344 -> 331127
5345 -> 521198
5346 -> 1277000
5347 -> 118691149
5348 -> 60907060
5349 -> 1395644
5350 -> 1290236
5351 -> 40009
5352 -> 109584
5353 -> 1305732
5354 -> 1074580
5355 -> 503987
5356 -> 432340
5357 -> 884313
5358 -> 421172
5359 -> 909023
5360 -> 642158
5361 -> 812567
5362 -> 1378475
5363 -> 880158


5828 -> 712339
5829 -> 1271281
5830 -> 135645
5831 -> 386828
5832 -> 1416765
5833 -> 1259448
5834 -> 813552
5835 -> 1475875
5836 -> 890984
5837 -> 1897504
5838 -> 1148873
5839 -> 1449398
5840 -> 788648
5841 -> 1050030
5842 -> 440533
5843 -> 1634345
5844 -> 1679791
5845 -> 481682
5846 -> 1739206
5847 -> 185862
5848 -> 980703
5849 -> 1915593
5850 -> 1970674
5851 -> 1317279
5852 -> 922705
5853 -> 767352
5854 -> 541057
5855 -> 664136
5856 -> 404801
5857 -> 1776043
5858 -> 770471
5859 -> 1688635
5860 -> 343724
5861 -> 1098298
5862 -> 1526289
5863 -> 42239274
5864 -> 533791
5865 -> 1986983
5866 -> 1585453
5867 -> 1356475
5868 -> 1606050
5869 -> 257199
5870 -> 1020927
5871 -> 480479
5872 -> 202663
5873 -> 1760469
5874 -> 759422
5875 -> 1659336
5876 -> 674170
5877 -> 1630657
5878 -> 522256
5879 -> 915856
5880 -> 1340322
5881 -> 441215
5882 -> 849002
5883 -> 1377194
5884 -> 1811660
5885 -> 106632
5886 -> 1233787
5887 -> 286342
5888 -> 840104
5889 -> 219622
5890 -> 580425
5891 -> 1556831
5892 ->

In [146]:
# Create a DataFrame to store the token along with it's id
token_sc_india_recent = pd.DataFrame(columns=['Document Id', 'Tokens'])
word_count_list = []

for x, y in enumerate(doc_list):
    doc_id = sc_india_recent['Document Id'][x]
    clean_doc = clean(y)    
    token_doc = spacy_tokenizer(clean_doc)
    
    print(f"{x} -> {sc_india_recent['Document Id'][x]}")
    
    token_sc_india_recent = token_sc_india_recent.append({ 'Document Id' : doc_id, 'Tokens' : token_doc}, ignore_index = True)
    word_count_list.append(word_count_dic(doc_id, token_doc))
    

0 -> 484625
1 -> 60864
2 -> 1373794
3 -> 55804
4 -> 301256
5 -> 342526
6 -> 315034
7 -> 952963
8 -> 1919674
9 -> 339039
10 -> 828091
11 -> 30721
12 -> 330864
13 -> 1871672
14 -> 1444315
15 -> 1070947
16 -> 171884
17 -> 1703868
18 -> 864906
19 -> 1087177
20 -> 88397
21 -> 577272
22 -> 952306
23 -> 1392246
24 -> 302447
25 -> 723319
26 -> 316662
27 -> 869424
28 -> 161755
29 -> 32069
30 -> 1072924
31 -> 581738
32 -> 492319
33 -> 1404079
34 -> 628924
35 -> 29716
36 -> 1796201
37 -> 526619
38 -> 1787248
39 -> 165700
40 -> 511928
41 -> 1573419
42 -> 1825956
43 -> 1058158
44 -> 754035
45 -> 1975354
46 -> 672732
47 -> 1981933
48 -> 108824706
49 -> 104106942
50 -> 1483568
51 -> 619652
52 -> 849832
53 -> 559864
54 -> 492051
55 -> 1466071
56 -> 1581335
57 -> 1622700
58 -> 124441393
59 -> 1815829
60 -> 1799063
61 -> 1974403
62 -> 404475
63 -> 96514934
64 -> 1269481
65 -> 465907
66 -> 1065048
67 -> 805009
68 -> 1178541
69 -> 48569
70 -> 696401
71 -> 849501
72 -> 1242489
73 -> 643598
74 -> 4960191
75

569 -> 441929
570 -> 1974324
571 -> 1206100
572 -> 1701224
573 -> 1917283
574 -> 1389589
575 -> 1791620
576 -> 1600445
577 -> 1766988
578 -> 605834
579 -> 1323993
580 -> 1483262
581 -> 1815324
582 -> 1706713
583 -> 641383
584 -> 870673
585 -> 989798
586 -> 592428
587 -> 454130
588 -> 1973429
589 -> 1130828
590 -> 53461189
591 -> 1030147
592 -> 406616
593 -> 840688
594 -> 1248667
595 -> 1691248
596 -> 455820
597 -> 1865386
598 -> 893566
599 -> 839959
600 -> 902306
601 -> 1442809
602 -> 1242357
603 -> 147864248
604 -> 253640
605 -> 1138910
606 -> 1945324
607 -> 1852811
608 -> 791266
609 -> 735843
610 -> 199374
611 -> 1752197
612 -> 739978
613 -> 162668
614 -> 250025
615 -> 97268
616 -> 1132839
617 -> 1679705
618 -> 1117516
619 -> 596603
620 -> 812915
621 -> 865927
622 -> 874542
623 -> 1439141
624 -> 1670628
625 -> 1210590
626 -> 135784847
627 -> 1725787
628 -> 656475
629 -> 143274
630 -> 1352674
631 -> 1665059
632 -> 795107
633 -> 515955
634 -> 1928163
635 -> 678728
636 -> 139810459
637 

1124 -> 1027994
1125 -> 586957
1126 -> 1737583
1127 -> 1479522
1128 -> 1299452
1129 -> 673494
1130 -> 139299795
1131 -> 137379
1132 -> 628651
1133 -> 841113
1134 -> 1928188
1135 -> 1918955
1136 -> 1085133
1137 -> 261589
1138 -> 1915429
1139 -> 176692787
1140 -> 276646
1141 -> 853433
1142 -> 1669097
1143 -> 1654170
1144 -> 1736083
1145 -> 1248305
1146 -> 1800100
1147 -> 892046
1148 -> 1890435
1149 -> 1949357
1150 -> 1919444
1151 -> 74318473
1152 -> 96789
1153 -> 433507
1154 -> 1277242
1155 -> 1013105
1156 -> 1241417
1157 -> 761005
1158 -> 149756
1159 -> 1808484
1160 -> 765135
1161 -> 40737166
1162 -> 104928945
1163 -> 196100928
1164 -> 783492
1165 -> 712499
1166 -> 747596
1167 -> 34445
1168 -> 1829378
1169 -> 428148
1170 -> 309619
1171 -> 202973
1172 -> 45961
1173 -> 325522
1174 -> 1904257
1175 -> 1386587
1176 -> 1014506
1177 -> 315007
1178 -> 1868789
1179 -> 1651003
1180 -> 1957714
1181 -> 18551101
1182 -> 50693703
1183 -> 1768336
1184 -> 255258
1185 -> 1824031
1186 -> 1944115
1187 -> 

1652 -> 76327630
1653 -> 1412571
1654 -> 144524555
1655 -> 1798667
1656 -> 1349336
1657 -> 1992311
1658 -> 1189438
1659 -> 208104
1660 -> 1234243
1661 -> 373674
1662 -> 1541035
1663 -> 1842561
1664 -> 637121
1665 -> 1765136
1666 -> 1257127
1667 -> 1386978
1668 -> 1670737
1669 -> 1444013
1670 -> 1020761
1671 -> 212701
1672 -> 1132466
1673 -> 103664
1674 -> 240136
1675 -> 1310828
1676 -> 1382201
1677 -> 181338271
1678 -> 317443
1679 -> 818332
1680 -> 1861993
1681 -> 1246989
1682 -> 296919
1683 -> 1622732
1684 -> 1493294
1685 -> 849428
1686 -> 200529
1687 -> 288213
1688 -> 816027
1689 -> 1877867
1690 -> 1318795
1691 -> 160887
1692 -> 48351
1693 -> 339045
1694 -> 100834628
1695 -> 1573754
1696 -> 1529194
1697 -> 1224351
1698 -> 1584515
1699 -> 1180876
1700 -> 1212086
1701 -> 554694
1702 -> 516669
1703 -> 1201968
1704 -> 854624
1705 -> 1724440
1706 -> 924107
1707 -> 711454
1708 -> 424832
1709 -> 1389120
1710 -> 1128474
1711 -> 1131810
1712 -> 886148
1713 -> 1315597
1714 -> 1028758
1715 -> 1

2178 -> 570393
2179 -> 976161
2180 -> 408620
2181 -> 344937
2182 -> 462447
2183 -> 1854196
2184 -> 215097
2185 -> 921223
2186 -> 226350
2187 -> 342770
2188 -> 1539576
2189 -> 829043
2190 -> 1180569
2191 -> 609016
2192 -> 30094296
2193 -> 1007126
2194 -> 1847321
2195 -> 1287030
2196 -> 1458298
2197 -> 1868108
2198 -> 186697
2199 -> 806787
2200 -> 1926604
2201 -> 852956
2202 -> 1801195
2203 -> 1200609
2204 -> 1295690
2205 -> 586656
2206 -> 25199
2207 -> 1754001
2208 -> 14169
2209 -> 839516
2210 -> 427559
2211 -> 137487
2212 -> 1132527
2213 -> 1090819
2214 -> 1978077
2215 -> 1806497
2216 -> 1292274
2217 -> 634300
2218 -> 1002182
2219 -> 248422
2220 -> 40205
2221 -> 234299
2222 -> 20185201
2223 -> 518973
2224 -> 1746530
2225 -> 127004
2226 -> 95490
2227 -> 1440508
2228 -> 770879
2229 -> 1209191
2230 -> 1200520
2231 -> 393527
2232 -> 1596666
2233 -> 308516
2234 -> 1232029
2235 -> 665015
2236 -> 1934517
2237 -> 1727179
2238 -> 696089
2239 -> 1257569
2240 -> 1814011
2241 -> 1687188
2242 -> 59

...

3735 -> 1546846
3736 -> 1728059
3737 -> 1676807
3738 -> 99628
3739 -> 905647
3740 -> 83284911
3741 -> 475850
3742 -> 1283428
3743 -> 300166
3744 -> 1724715
3745 -> 143699
3746 -> 1181002
3747 -> 146444
3748 -> 14299467
3749 -> 62010481
3750 -> 1295003
3751 -> 1629851
3752 -> 619838
3753 -> 1167322
3754 -> 65801028
3755 -> 976548
3756 -> 146455
3757 -> 1550126
3758 -> 301898
3759 -> 1938735
3760 -> 1583561
3761 -> 1943124
3762 -> 789051
3763 -> 1740690
3764 -> 110957682
3765 -> 761884
3766 -> 955330
3767 -> 1598563
3768 -> 38394
3769 -> 1870380
3770 -> 1549187
3771 -> 449895
3772 -> 520092
3773 -> 1640713
3774 -> 1097981
3775 -> 434621
3776 -> 1812977
3777 -> 622430
3778 -> 255210
3779 -> 1419648
3780 -> 1660548
3781 -> 350207
3782 -> 1498806
3783 -> 1976638
3784 -> 354829
3785 -> 686974
3786 -> 752130
3787 -> 851066
3788 -> 548162
3789 -> 943151
3790 -> 1332816
3791 -> 512571
3792 -> 870905
3793 -> 1582820
3794 -> 1957755
3795 -> 81931
3796 -> 709616
3797 -> 98822
3798 -> 1819336
3799 

4258 -> 33643539
4259 -> 421171
4260 -> 74242914
4261 -> 44195342
4262 -> 1111437
4263 -> 185121721
4264 -> 139424125
4265 -> 129292798
4266 -> 105430
4267 -> 1059476
4268 -> 1876565
4269 -> 321019
4270 -> 1418721
4271 -> 1704181
4272 -> 1445805
4273 -> 231341
4274 -> 1290534
4275 -> 19951
4276 -> 122509
4277 -> 1293440
4278 -> 573333
4279 -> 1570352
4280 -> 1916813
4281 -> 1290723
4282 -> 1923755
4283 -> 1069885
4284 -> 511501
4285 -> 1216036
4286 -> 1951229
4287 -> 1084188
4288 -> 1497720
4289 -> 627760
4290 -> 1684919
4291 -> 225599
4292 -> 1783984
4293 -> 160789
4294 -> 1612780
4295 -> 1926027
4296 -> 1867627
4297 -> 496676
4298 -> 1618302
4299 -> 853716
4300 -> 1515604
4301 -> 492377
4302 -> 196302305
4303 -> 176647548
4304 -> 1072861
4305 -> 985284
4306 -> 766886
4307 -> 1615098
4308 -> 124279
4309 -> 853784
4310 -> 1721619
4311 -> 80804
4312 -> 240789
4313 -> 634266
4314 -> 441874
4315 -> 1864006
4316 -> 1667245
4317 -> 1992014
4318 -> 1825409
4319 -> 1910287
4320 -> 451460
4321

4784 -> 803726
4785 -> 298579
4786 -> 765792
4787 -> 770272
4788 -> 648736
4789 -> 841129
4790 -> 1778819
4791 -> 1068747
4792 -> 161861178
4793 -> 947
4794 -> 712234
4795 -> 1436672
4796 -> 1332409
4797 -> 748814
4798 -> 1628531
4799 -> 982131
4800 -> 1583669
4801 -> 1203458
4802 -> 1423669
4803 -> 1732268
4804 -> 518831
4805 -> 1924157
4806 -> 607625
4807 -> 1147175
4808 -> 1994838
4809 -> 855361
4810 -> 970620
4811 -> 1729858
4812 -> 1177323
4813 -> 168545311
4814 -> 210097
4815 -> 49133735
4816 -> 1756632
4817 -> 1265820
4818 -> 348899
4819 -> 196226299
4820 -> 1501179
4821 -> 558373
4822 -> 149012918
4823 -> 1604025
4824 -> 818083
4825 -> 119042
4826 -> 198772887
4827 -> 64772
4828 -> 264557
4829 -> 150881
4830 -> 617135
4831 -> 1198044
4832 -> 1787277
4833 -> 1310174
4834 -> 1838507
4835 -> 12214027
4836 -> 342073
4837 -> 1409685
4838 -> 1300862
4839 -> 1463985
4840 -> 580863
4841 -> 1661743
4842 -> 1253073
4843 -> 1056962
4844 -> 586838
4845 -> 1820269
4846 -> 1766285
4847 -> 80

5306 -> 921001
5307 -> 790630
5308 -> 866941
5309 -> 1074322
5310 -> 910940
5311 -> 1357418
5312 -> 1534452
5313 -> 1589069
5314 -> 73715021
5315 -> 182991489
5316 -> 748067
5317 -> 1594596
5318 -> 1189023
5319 -> 833158
5320 -> 1561222
5321 -> 942439
5322 -> 1972239
5323 -> 899528
5324 -> 1915099
5325 -> 301445
5326 -> 527656
5327 -> 1430296
5328 -> 852405
5329 -> 1221178
5330 -> 115310075
5331 -> 1617880
5332 -> 391625
5333 -> 1338744
5334 -> 1758778
5335 -> 649814
5336 -> 948361
5337 -> 1897303
5338 -> 1497806
5339 -> 1951051
5340 -> 1078624
5341 -> 1527139
5342 -> 1184356
5343 -> 1393826
5344 -> 331127
5345 -> 521198
5346 -> 1277000
5347 -> 118691149
5348 -> 60907060
5349 -> 1395644
5350 -> 1290236
5351 -> 40009
5352 -> 109584
5353 -> 1305732
5354 -> 1074580
5355 -> 503987
5356 -> 432340
5357 -> 884313
5358 -> 421172
5359 -> 909023
5360 -> 642158
5361 -> 812567
5362 -> 1378475
5363 -> 880158
5364 -> 1405448
5365 -> 1563218
5366 -> 1985739
5367 -> 83298
5368 -> 1791396
5369 -> 58708

5833 -> 1259448
5834 -> 813552
5835 -> 1475875
5836 -> 890984
5837 -> 1897504
5838 -> 1148873
5839 -> 1449398
5840 -> 788648
5841 -> 1050030
5842 -> 440533
5843 -> 1634345
5844 -> 1679791
5845 -> 481682
5846 -> 1739206
5847 -> 185862
5848 -> 980703
5849 -> 1915593
5850 -> 1970674
5851 -> 1317279
5852 -> 922705
5853 -> 767352
5854 -> 541057
5855 -> 664136
5856 -> 404801
5857 -> 1776043
5858 -> 770471
5859 -> 1688635
5860 -> 343724
5861 -> 1098298
5862 -> 1526289
5863 -> 42239274
5864 -> 533791
5865 -> 1986983
5866 -> 1585453
5867 -> 1356475
5868 -> 1606050
5869 -> 257199
5870 -> 1020927
5871 -> 480479
5872 -> 202663
5873 -> 1760469
5874 -> 759422
5875 -> 1659336
5876 -> 674170
5877 -> 1630657
5878 -> 522256
5879 -> 915856
5880 -> 1340322
5881 -> 441215
5882 -> 849002
5883 -> 1377194
5884 -> 1811660
5885 -> 106632
5886 -> 1233787
5887 -> 286342
5888 -> 840104
5889 -> 219622
5890 -> 580425
5891 -> 1556831
5892 -> 576851
5893 -> 1145437
5894 -> 199696
5895 -> 453820
5896 -> 641069
5897 -> 

In [147]:
# Check the list and dataframe created
print(f'Dictionary list of length {len(word_count_list)}')
print(f'DataFrame shape {token_sc_india_recent.shape}')

print(f"Document Id at the start of the reading {word_count_list[0].keys()} == {token_sc_india_recent.loc[0]['Document Id']}")
print(f"Document Id at the end of the reading {word_count_list[token_sc_india_recent.shape[0]-1].keys()} == {token_sc_india_recent.loc[token_sc_india_recent.shape[0]-1]['Document Id']}")

Dictionary list of length 6030
DataFrame shape (6030, 2)
Document Id at the start of the reading dict_keys([484625]) == 484625
Document Id at the end of the reading dict_keys([1223961]) == 1223961


In [148]:
# Save the list in a pickle file
pickle_on = open("word_count_list.pickle","wb")
pickle.dump(word_count_list, pickle_on)
pickle_on.close()

In [149]:
# Save the dataframe in csv file
token_sc_india_recent.to_csv('Supreme_Court_cases_India_tokens_above_1999.csv')

In [56]:
# Check the list and dataframe is stored properly
with open('word_count_list.pickle', 'rb') as f:
    store_word_count_1 = pickle.load(f)
    
sc_india_recent_stored_1 = pd.read_csv('Supreme_Court_cases_India_tokens_above_1999.csv')
sc_india_recent_stored_1.drop(sc_india_recent_stored_1.filter(regex="Unname"),axis=1, inplace=True)
print(f'Dictionary list of length {len(store_word_count_1)}')
print(f'DataFrame shape {sc_india_recent_stored_1.shape}')

print(f"Document Id at the start of the reading {store_word_count_1[0].keys()} == {sc_india_recent_stored_1.loc[0]['Document Id']}")
print(f"Document Id at the end of the reading {store_word_count_1[sc_india_recent_stored_1.shape[0]-1].keys()} == {sc_india_recent_stored_1.loc[sc_india_recent_stored_1.shape[0]-1]['Document Id']}")

Dictionary list of length 6030
DataFrame shape (6030, 2)
Document Id at the start of the reading dict_keys([484625]) == 484625
Document Id at the end of the reading dict_keys([1223961]) == 1223961


In [52]:
# store_word_count[6029]

In [57]:
# Check the list and dataframe is stored properly
with open('word_count_list_2006.pickle', 'rb') as f:
    store_word_count_2 = pickle.load(f)
    
sc_india_recent_stored_2 = pd.read_csv('Supreme_Court_cases_India_tokens_above_2006.csv')
sc_india_recent_stored_2.drop(sc_india_recent_stored_2.filter(regex="Unname"),axis=1, inplace=True)

print(f'Dictionary list of length {len(store_word_count_2)}')
print(f'DataFrame shape {sc_india_recent_stored_2.shape}')

print(f"Document Id at the start of the reading {store_word_count_2[0].keys()} == {sc_india_recent_stored_2.loc[0]['Document Id']}")
print(f"Document Id at the end of the reading {store_word_count_2[sc_india_recent_stored_2.shape[0]-1].keys()} == {sc_india_recent_stored_2.loc[sc_india_recent_stored_2.shape[0]-1]['Document Id']}")

Dictionary list of length 14239
DataFrame shape (14239, 2)
Document Id at the start of the reading dict_keys([1903089]) == 1903089
Document Id at the end of the reading dict_keys([34010735]) == 34010735


In [6]:
sc_india_recent = sc_india[sc_india.Year >= 2001].reset_index()  
sc_india_recent.head()

,index,Case Title,Year,Author,Judges,Document Id
0,28416,Smt. Kaushnuma Begum And Ors vs The New India ...,2001,NaN,"K.T.Thomas, R.P.Sethi",484625
1,28417,Dalmia Cement (Bharat) Ltd vs M/S.Galaxy Trade...,2001,Sethi,"R.P.Sethi, K.T.Thomas",60864
2,28418,Suresh Chand Jain vs State Of Madhya Pradesh &...,2001,NaN,"K.T.Thomas, R.P.Sethi",1373794
3,28419,West Bengal State Electricity ... vs Patel Eng...,2001,S S Quadri,"S.S.M.Quadri, S.N.Phukan",55804
4,28420,Ragu Thilak D. John vs S.Rayappan & Others on ...,2001,Sethi,"K.T. Thomas, R.P. Sethi.",301256


In [9]:
sc_india_recent[6030-2:6030+3]

,index,Case Title,Year,Author,Judges,Document Id
6028,34444,Madan Lal And Anr. vs Bal Krishan And Ors. on ...,2005,A Pasayat,"A Pasayat, T Chatterjee",467623
6029,34445,A. Maimoona vs State Of Tamil Nadu & Ors on 16...,2005,P Balasubramanyan,"S.B. Sinha, P.K. Balasubramanyan",1223961
6030,34446,"Mayar (H.K.) Ltd. & Ors vs Owners & Parties, V...",2006,P Naolekar,"Ruma Pal, P.P. Naolekar",1903089
6031,34447,Sandvik Asia Ltd vs Commissioner Of Income Tax...,2006,. A Lakshmanan,"H.K. Sema, Dr. Ar. Lakshmanan",287995
6032,34448,Mohd. Yousuf vs Smt. Afaq Jahan & Anr on 2 Jan...,2006,A Pasayat,"Arijit Pasayat, S.H. Kapadia",1938541


In [12]:
sc_india_recent[20268-2:20268+3]

,index,Case Title,Year,Author,Judges,Document Id
20266,48682,Raghavendra Swamy Mutt vs Uttaradi Mutt on 30 ...,2016,NaN,"Dipak Misra, Shiva Kirti Singh",160092465
20267,48683,Siddharth Chaturvedi vs Securities And Exchang...,2016,........................J.,"Kurian Joseph, Rohinton Fali Nariman",88547322
20268,48684,State Of Kerala vs P.B.Sourabhan & Ors on 4 Ma...,2016,R Gogoi,"Ranjan Gogoi, Prafulla C. Pant",34010735
20269,48685,"Hardei vs State Of U.P on 30 March, 2016",2016,S K Singh,"Dipak Misra, Shiva Kirti Singh",30534286
20270,48686,D Sudhakar vs State Of A P And Ors on 28 March...,2016,NaN,"Madan B. Lokur, S.A. Bobde",142246040


In [ ]:
# Documents after 2016 are remaining to be fetched

In [58]:
# Append two dataframe
sc_india_recent_stored = sc_india_recent_stored_1.append(sc_india_recent_stored_2, ignore_index=True)
sc_india_recent_stored.drop(sc_india_recent_stored.filter(regex="Unname"),axis=1, inplace=True)
sc_india_recent_stored.shape

(20269, 2)

In [59]:
sc_india_recent_stored.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20269 entries, 0 to 20268
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Document Id  20269 non-null  int64 
 1   Tokens       20269 non-null  object
dtypes: int64(1), object(1)
memory usage: 316.8+ KB


In [23]:
sc_india_recent_stored['Corpus'] = sc_india_recent_stored.Tokens.apply(lambda x :  re.sub(r'\[|\]|\'|\,', '',x))

In [24]:
sc_india_recent_stored.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20269 entries, 0 to 20268
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Document Id  20269 non-null  int64 
 1   Tokens       20269 non-null  object
 2   Corpus       20269 non-null  object
dtypes: int64(1), object(2)
memory usage: 475.2+ KB


In [26]:
sc_india_recent_stored.head()

,Document Id,Tokens,Corpus
0,484625,"['littttttj', 'maintainedbeforethe', 'motor', ...",littttttj maintainedbeforethe motor accident t...
1,60864,"['littttttj', 'sethijthe', 'complaint', 'thene...",littttttj sethijthe complaint thenegotiable in...
2,1373794,"['complaint', 'forwardedby', 'magistrate', 'po...",complaint forwardedby magistrate police regist...
3,55804,"['littttttj', 'sye', 'quadri', 'jthis', 'benga...",littttttj sye quadri jthis bengal electricity ...
4,301256,"['litttttttjsethijdelay', 'condonedleave', 'gr...",litttttttjsethijdelay condonedleave grantedthe...


In [27]:
# Save the dataframe in csv file
sc_india_recent_stored.to_csv('Supreme_Court_cases_India_tokens_above_1999_final.csv')

In [28]:
# Append the word_count list

In [60]:
len(store_word_count_1)

6030

In [61]:
len(store_word_count_2)

14239

In [62]:
store_word_count_final = store_word_count_1 + store_word_count_2
len(store_word_count_final)

20269

In [63]:
# Save the list in a pickle file
pickle_on = open("word_count_list_final.pickle","wb")
pickle.dump(store_word_count_final, pickle_on)
pickle_on.close()

In [65]:
# Conclusion - A dataframe is created that contains a tokenized words and corpus along with document Id
# We are also maintaining a document dictionary that conatins the frequency of the word.